In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import segmentation, color
import csv
from skimage.feature import graycomatrix, graycoprops
from skimage import io, color, img_as_ubyte
import skimage.util as util
from skimage.metrics import structural_similarity as ssim

C:\Users\krist\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [2]:
Lesion_images_folder="C:\\Users\\krist\\Desktop\\ITU\\2_Semester\\projects_in_datascience\\first_year_project\\groupN_images"
Lesion_masks_folder="C:\\Users\\krist\\Desktop\\ITU\\2_Semester\\projects_in_datascience\\first_year_project\\groupN_masks"
Metadata_path="C:\\Users\\krist\\Desktop\\ITU\\2_Semester\\projects_in_datascience\\first_year_project\\metadata.csv"

In [3]:
def measure_symmetry(mask_path, image_path):
    """
    Measure symmetry of a binary mask and color similarity within the masked area.

    Args:
        mask_path (str): Path to the binary mask image file.
        image_path (str): Path to the original image corresponding to the mask.

    Returns:
        float: Symmetry score between 0 and 1, where 1 indicates perfect symmetry.
        float: Color-based symmetry score based on color similarity within the masked area.
    """
    # Load the binary mask image
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    
    # Load the original image corresponding to the mask
    original_image = cv2.imread(image_path)

    # Calculate the total area (number of non-zero pixels) of the mask
    area_total = np.count_nonzero(mask)

    if area_total == 0:
        return 0.0, 0.0  # If the mask is empty, return symmetry scores of 0

    # Compute major and minor axes using PCA (Principal Component Analysis)
    masked_pixels = np.transpose(np.nonzero(mask))
    com_x, com_y = np.mean(masked_pixels, axis=0)
    cov_matrix = np.cov(masked_pixels, rowvar=False)
    _, eigenvectors = np.linalg.eigh(cov_matrix)
    pc1_x, pc1_y = eigenvectors[:, 0]  # Major axis
    pc2_x, pc2_y = eigenvectors[:, 1]  # Minor axis

    # Flip mask over major and minor axes
    mask_major_axis = np.fliplr(mask)  # Horizontal flip (major axis)
    mask_minor_axis = np.flipud(mask)  # Vertical flip (minor axis)

    # Calculate intersection areas between original mask and flipped versions
    intersection_major = mask & mask_major_axis
    intersection_minor = mask & mask_minor_axis

    # Calculate symmetry scores based on intersection areas
    symmetry_major = np.count_nonzero(intersection_major) / area_total
    symmetry_minor = np.count_nonzero(intersection_minor) / area_total

    # Calculate overall symmetry score (average of major and minor axes)
    symmetry_score = 1 - 0.5 * (symmetry_major + symmetry_minor)

    # Extract masked regions from the original image
    masked_region = original_image[mask > 0]
    masked_major_axis = original_image[mask_major_axis > 0]
    masked_minor_axis = original_image[mask_minor_axis > 0]

    # Calculate SSIM (Structural Similarity Index) for color similarity
    ssim_major = ssim(masked_region, masked_major_axis, win_size=min(masked_region.shape[0], masked_region.shape[1]), multichannel=True)
    ssim_minor = ssim(masked_region, masked_minor_axis, win_size=min(masked_region.shape[0], masked_region.shape[1]), multichannel=True)

    # Calculate color-based symmetry score (average SSIM)
    color_symmetry_score = 1 - 0.5 * (ssim_major + ssim_minor)
    
    temp_sym_major = (1 if 1-symmetry_major <= 0.5 else 2)
    temp_sym_minor = (1 if 1-symmetry_minor <= 0.5 else 2)
    
    temp_col_major = (1 if 1-ssim_major <= 0.5 else 2)
    temp_col_minor = (1 if 1-ssim_minor <= 0.5 else 2)
    
    return [temp_sym_major, temp_col_major, temp_sym_minor, temp_col_minor]

In [42]:
import cv2
import numpy as np
import os

def calculate_lesion_score(image_path, mask_path):
    # Load the image
    image = cv2.imread(image_path)

    # Convert the image to the color space that you will use for color detection
    # For example, HSV
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Define color ranges for each color
    color_ranges = {
        'white': ((200, 200, 200), (255, 255, 255)),  # White color range in RGB
        'red': ((0, 0, 150), (100, 100, 255)),  # Red color range in RGB
        'light_brown': ((150, 100, 50), (200, 150, 100)),  # Light brown color range in RGB
        'dark_brown': ((50, 30, 10), (100, 70, 40)),  # Dark brown color range in RGB
        'blue_green': ((0, 100, 100), (50, 180, 150)),  # Blue-green color range in RGB
        'black': ((0, 0, 0), (50, 50, 50))  # Black color range in RGB
    }


    # Load the mask
    mask = cv2.imread(mask_path, 0)

    # Initialize the score
    score = 0
    detected_colors = []

    # Check each color's presence within the lesion and increment the score
    for color, (lower, upper) in color_ranges.items():
        color_mask = cv2.inRange(image_rgb, lower, upper)
        if np.any(cv2.bitwise_and(color_mask, color_mask, mask=mask)):
            score += 1
            detected_colors.append(color)
    
    # The order of the colors in the results array
    color_order = ['white', 'red', 'light_brown', 'dark_brown', 'blue_gray', 'black']
    temp = []
    for i in color_order:
        if i in detected_colors:
            temp.append(2)
        else:
            temp.append(1)
            
    return temp

In [43]:
Colour_results = {}
# Loop through each image in the folder
for filename in os.listdir(Lesion_images_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # Adjust file extensions as needed
        img_path = os.path.join(Lesion_images_folder, filename)

        # Find corresponding mask
        corresponding_mask = os.path.join(Lesion_masks_folder, os.path.basename(os.path.splitext(img_path)[0]) + "_mask.png")
        # Analyze and visualize segmentation
        result = calculate_lesion_score(img_path, corresponding_mask);
        
        # Append results to Colour_results list
        ID=filename.replace(".png","");
        Colour_results[ID]=result;

In [44]:
Colour_results

{'PAT_1000_31_620': [1, 1, 2, 2, 1, 1],
 'PAT_101_1041_658': [2, 1, 2, 2, 1, 2],
 'PAT_1021_112_40': [1, 1, 2, 1, 1, 1],
 'PAT_1022_114_756': [1, 1, 2, 1, 1, 1],
 'PAT_1026_124_346': [1, 1, 1, 1, 1, 1],
 'PAT_1063_270_593': [1, 1, 2, 1, 1, 1],
 'PAT_1064_272_668': [2, 1, 1, 1, 1, 2],
 'PAT_1071_312_841': [1, 1, 2, 1, 1, 1],
 'PAT_1092_378_922': [1, 1, 2, 1, 1, 1],
 'PAT_1102_408_613': [1, 1, 2, 1, 1, 1],
 'PAT_114_173_213': [2, 1, 2, 2, 1, 2],
 'PAT_1174_628_915': [1, 1, 1, 1, 1, 1],
 'PAT_120_183_623': [2, 1, 2, 2, 1, 1],
 'PAT_1244_840_975': [2, 1, 2, 1, 1, 1],
 'PAT_1277_964_77': [2, 1, 1, 2, 1, 2],
 'PAT_1306_1086_124': [1, 1, 1, 2, 1, 2],
 'PAT_1309_1096_393': [1, 1, 1, 1, 1, 1],
 'PAT_1317_1130_656': [2, 1, 2, 1, 1, 1],
 'PAT_132_198_114': [2, 1, 2, 1, 1, 1],
 'PAT_1331_1173_380': [1, 1, 1, 1, 1, 1],
 'PAT_1371_1278_533': [1, 1, 1, 2, 1, 1],
 'PAT_1374_1286_775': [1, 1, 2, 1, 1, 1],
 'PAT_1418_1448_307': [1, 1, 2, 1, 1, 1],
 'PAT_141_209_438': [1, 1, 2, 1, 1, 1],
 'PAT_1427_1485_

In [7]:
Asymmetry_results={}
# Loop through each image in the folder
for filename in os.listdir(Lesion_images_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # Adjust file extensions as needed
        image_path = os.path.join(Lesion_images_folder, filename)
        mask_path = os.path.join(Lesion_masks_folder, os.path.basename(os.path.splitext(image_path)[0]) + "_mask.png")
        symmetry = measure_symmetry(mask_path, image_path)
        ID = filename.replace(".png","");
        Asymmetry_results[ID]= symmetry
#Asymmetry_results      


In [22]:
Combine = {}

for key in Asymmetry_results:
    if key in Colour_results:
        Combine[key] = (Asymmetry_results[key] + Colour_results[key]) + [1]

In [30]:
import pandas as pd

# Example DataFrame (Replace this with your DataFrame 'df')


import pandas as pd


# Create a DataFrame from the dictionary
df = pd.DataFrame(list(Combine.items()), columns=["image", "values"])
# Display the DataFrame
# print(df)

# Explode the 'values' list into separate columns
expanded_values = pd.DataFrame(df['values'].tolist(), columns=["symmetry_major", "ssim_major", "symmetry_minor", "ssim_minor", "white", "red", "light brown","dark brown", "green blue", "black","blue_white_veil_score"]
)

# Concatenate the expanded values DataFrame with the original DataFrame
df_expanded = pd.concat([df['image'], expanded_values], axis=1)


In [31]:
print(df_expanded)

                image  symmetry_major  ssim_major  symmetry_minor  ssim_minor  \
0     PAT_1000_31_620               1           1               1           1   
1    PAT_101_1041_658               1           1               1           1   
2     PAT_1021_112_40               1           1               2           1   
3    PAT_1022_114_756               2           1               2           1   
4    PAT_1026_124_346               2           1               2           1   
..                ...             ...         ...             ...         ...   
121  PAT_978_1844_715               1           1               1           1   
122  PAT_990_1860_636               1           1               1           1   
123  PAT_994_1866_730               1           1               1           1   
124    PAT_998_17_641               1           1               1           1   
125       PAT_9_17_80               1           1               1           1   

     white  red  light brow

In [33]:
# Specify the file path where you want to save the CSV file
csv_file_path = 'computer_annotation3.csv'

# Save the DataFrame to a CSV file
df_expanded.to_csv(csv_file_path, index=False)

print(f"DataFrame has been saved to '{csv_file_path}' as a CSV file.")

DataFrame has been saved to 'computer_annotation3.csv' as a CSV file.
